In [1]:
import sys
sys.path.append("..") 

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import json
import torch
import torch.nn.functional as F
import random
import numpy as np
import tqdm as notebook_tqdm
import clip
import argparse
import yaml
from dotmap import DotMap
from modules.video_clip import video_header, VideoCLIP
import torch.backends.cudnn as cudnn
import torch.distributed as dist

from utils.Augmentation import get_augmentation
from utils.utils import init_distributed_mode, AverageMeter, reduce_tensor, accuracy

import torch

from datasets import Video_dataset
from torch.utils.data import DataLoader

import tome
from tome.patch.clip import patch_openclip

/opt/conda/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libpng16.so.16: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_parser():
    parser = argparse.ArgumentParser()
    # parser.add_argument('--config', '-cfg', type=str, default='clip.yaml', help='global config file')
    parser.add_argument('--log_time', default='001')
    parser.add_argument('--dist_url', default='env://',
                        help='url used to set up distributed training')
    parser.add_argument('--world_size', default=1, type=int,
                        help='number of distributed processes')                        
    parser.add_argument("--local_rank", type=int,
                        help='local rank for DistributedDataParallel')
    parser.add_argument(
        "--precision",
        choices=["amp", "fp16", "fp32"],
        default="amp",
        help="Floating point precition."
    )
    parser.add_argument(
        "--master_port",
        default=1027
    )
    parser.add_argument(
        "--nproc_per_node",
        default=1
    )                        
    # args = parser.parse_args()
    args = parser.parse_known_args()[0]
    return args


In [3]:
args = get_parser()
init_distributed_mode(args)

Not using distributed mode


In [4]:
with open('/home/code/lyx/tome/configs/hmdb51/hmdb_few_shot.yaml', 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
config = DotMap(config)

if torch.cuda.is_available():
        device = "cuda"
        cudnn.benchmark = True
        
# # fix the seed for reproducibility
# seed = config.seed + dist.get_rank()
# torch.manual_seed(seed)
# np.random.seed(seed)

In [5]:
# get fp16 model and weight
model, clip_state_dict = clip.load(
    config.network.arch,
    device="cpu",jit=False,
    internal_modeling=config.network.tm,
    T=config.data.num_segments,
    dropout=config.network.drop_out,
    emb_dropout=config.network.emb_dropout,
    pretrain=config.network.init,
    joint_st=config.network.joint_st) # Must set jit=False for training

In [6]:
classes = ["dog", "cat", "wolf", "people", "stone", "grass"]
text_dict = torch.cat([clip.tokenize(c) for c in classes])
model.eval()
with torch.no_grad():
    classes_features = model.encode_text(text_dict)  # [n_class dim]

In [7]:

video_head = video_header(
    config.network.sim_header,
    clip_state_dict)


model_full = VideoCLIP(model, video_head, config.data.num_segments)
model_full = model_full.to(device)

layer=6


In [8]:
device = "cuda"
runs = 50
batch_size = 4  # Lower this if you don't have that much memory
input_size = [config.data.num_segments * batch_size, 3, 224,224]

In [9]:
# baseline benchmark
model_full = model_full.float()
classes_features = classes_features.to(device)
baseline_throughput = tome.utils.benchmark(
    model_full,
    device=device,
    verbose=True,
    runs=runs,
    batch_size=batch_size,
    input_size=input_size,
    text = classes_features
)

Benchmarking: 100%|██████████| 50/50 [00:10<00:00,  4.64it/s]


Throughput: 5.70 im/s


In [10]:
patch_openclip(model, 16)

In [11]:
video_head = video_header(
    config.network.sim_header,
    clip_state_dict)


model_full = VideoCLIP(model, video_head, config.data.num_segments)
model_full = model_full.to(device)

layer=6


In [12]:
# Tome benchmark
model_full = model_full.float()
classes_features = classes_features.to(device)
tome_throughput = tome.utils.benchmark(
    model_full,
    device=device,
    verbose=True,
    runs=runs,
    batch_size=batch_size,
    input_size=input_size,
    text = classes_features
)
print(f"Throughput improvement: {tome_throughput / baseline_throughput:.2f}x")

Benchmarking: 100%|██████████| 50/50 [00:04<00:00, 11.39it/s]

Throughput: 15.55 im/s
Throughput improvement: 2.73x
